# A Model for the Human Genome
This example shows a simple simulation of 24 block copolymers that is a potential model of the Human Genome in hoomdd.

The human genome consists of 24 chromosomes with a total size of 3Gb.  Aiden et al. 2014 produced a 1kB resolution (below single gene resolution) Genome Contact Map.  Contact domains (∼185 kb) segregate into six nuclear subcompartments with distinct histone marks.
This can be reproduced phenomenologically with this simple block copolymer model.

from: https://lost-contact.mit.edu/afs//umich.edu/user/j/o/joaander/Public/hoomd-web/doc/page_example_scripts.html

## Initialize
First import hoomd and associated libraries and then initialize.  If no mode is given then the fastest processor cpu or gpu will be selected automatically.

In [1]:
#import hoomd
import hoomd.md
from hoomd import *
from hoomd.deprecated import *
#context.initialize("--mode=cpu");
context.initialize("--mode=gpu");

HOOMD-blue 2.2.4-unknown CUDA (8.0) DOUBLE HPMC_MIXED SSE SSE2 SSE3 SSE4_1 SSE4_2 AVX AVX2 
Compiled: 03/11/2018
Copyright 2009-2017 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
notice(2): This system is not compute exclusive, using local rank to select GPUs
HOOMD-blue is running on the following GPU(s):
 [0]   GeForce GTX 1050 Ti   6 SM_6.1 @ 1.39 GHz, 4095 MiB DRAM, DIS


## Import other libraries and define parameters

The human genome has 24 chromasomes.

In [2]:
import math
# parameters
phi_P = 0.25
n_poly = 24

## Define the polymer
The average length of a human chromosome is 133 Mb

In [3]:
3200/24 #Mb

133

So if we assume that Euchromatin makes up 60% of the genome and heterochromatin makes up 40% then a block copolymer of 1000 beads would have a resolution of approximately 133 kb/bead.

In [4]:
polymer1 = dict(bond_len=1.2, type=['A']*300 + ['B']*400 + ['A']*300,bond="linear", count=n_poly)
# perform some simple math to find the length of the box
N = len(polymer1['type']) * polymer1['count']
# generate the polymer system
init.create_random_polymers(box=data.boxdim(volume=10*math.pi * N / (6.0 * phi_P)), polymers=[polymer1],separation=dict(A=0.35, B=0.35),seed=12)


notice(2): Group "all" created containing 24000 particles


## Setup the bonds and force fields

In [5]:
# force field setup
harmonic = hoomd.md.bond.harmonic()
harmonic.bond_coeff.set('polymer', k=330.0, r0=0.84)
nl = hoomd.md.nlist.cell();
lj = hoomd.md.pair.lj(r_cut=3.0,nlist=nl)

lj.pair_coeff.set('A', 'A', epsilon=1.0, sigma=1.0, alpha=0.0,r_cut=2**(1.0/6.0))
lj.pair_coeff.set('A', 'B', epsilon=1.0, sigma=1.0, alpha=0.0,r_cut=2**(1.0/6.0))
lj.pair_coeff.set('B', 'B', epsilon=1.0, sigma=1.0, alpha=1.0,r_cut=3.0)
lj.set_params(mode='shift')
                  


## Integrate the simulation

In [ ]:
all = group.all()
# integrate NVT for a bunch of time steps
hoomd.md.integrate.mode_standard(dt=0.005)
hoomd.md.integrate.nvt(group=all, kT=1.2, tau=0.5)
#hoomd.md.integrate.brownian(group=all, kT=1.2, seed=1)

The results of this example may be visualized with VMD which can be installed from here: http://www.ks.uiuc.edu/Research/vmd/

In [ ]:
hoomd.dump.gsd(filename="gsd/hum_gen_24x1000.gsd", overwrite=True, period=None, group=group.all(), time_step=0)
# setup the IMD server
hoomd.analyze.imd(port=54321, period=10)
# run a very long time so the simulation can be watched in VMD
run(1e6)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 1 exclusions             : 48
notice(2): Particles with 2 exclusions             : 23952
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
notice(2): analyze.imd: listening on port 54321
Time 00:00:10 | Step 4671 / 1000000 | TPS 467.059 | ETA 00:35:31
Time 00:00:20 | Step 9532 / 1000000 | TPS 485.908 | ETA 00:33:58
Time 00:00:30 | Step 13861 / 1000000 | TPS 432.871 | ETA 00:37:58
notice(2): analyze.imd: accepted connection
Time 00:00:40 | Step 16829 / 1000000 | TPS 296.788 | ETA 00:55:12
Time 00:00:50 | Step 19177 / 1000000 | TPS 234.763 | ETA 01:09:37
Time

The results of this example may be visualized with VMD which can be installed from here: http://www.ks.uiuc.edu/Research/vmd/

While running must execute the following command in a terminal:

In [ ]:
#vmd -e imd2.vmd
!cat imd2.vmd

![](snapshots/hum_gen_100kb.png)